In [1]:
# dependencies BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pymongo
import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\jjel0\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


### NASA Mars News

In [3]:
# Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text.
    # Assign the text to variables that you can reference later.
url_mars_news = 'https://redplanetscience.com/'
browser.visit(url_mars_news)
html_mars_news = browser.html

In [4]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_news = bs(html_mars_news, 'html.parser')

# Print formatted version of the soup
print(soup_mars_news.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [5]:
# Collect the latest News Title and Paragraph Text.
list_text_list = soup_mars_news.find_all('div', class_='list_text')

content_title_strip_list = []
article_teaser_body_strip_list = []

# Loop through returned results (from 12-2-04)
for result in list_text_list:
    
    # Retrieve the title
    content_title = result.find('div', class_='content_title')
    content_title_strip = content_title.text.lstrip()
    print(f'news_title = "' + content_title_strip + '"')
    content_title_strip_list.append(content_title_strip)
    
    # Retrieve the article body
    article_teaser_body = result.find('div', class_='article_teaser_body')
    article_teaser_body_strip = article_teaser_body.text.lstrip()
    print(f'news_p = "' + article_teaser_body_strip + '"')
    article_teaser_body_strip_list.append(article_teaser_body_strip)
    print ('********************')

news_title = "NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network"
news_p = "Robotic spacecraft will be able to communicate with the dish using radio waves and lasers."
********************
news_title = "NASA Mars Mission Connects With Bosnian and Herzegovinian Town"
news_p = "A letter from NASA was presented to the mayor of Jezero, Bosnia-Herzegovina, honoring the connection between the town and Jezero Crater, the Mars 2020 rover landing site."
********************
news_title = "Global Storms on Mars Launch Dust Towers Into the Sky"
news_p = "A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze."
********************
news_title = "NASA's Curiosity Takes Selfie With 'Mary Anning' on the Red Planet"
news_p = "The Mars rover has drilled three samples of rock in this clay-enriched region since arriving in July."
*********

### JPL Mars Space Images - Featured Image

In [6]:
# Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
url_mars_imag = 'https://spaceimages-mars.com'
browser.visit(url_mars_imag)
html_mars_imag = browser.html

In [7]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_imag = bs(html_mars_imag, 'html.parser')

In [8]:
# Print formatted version of the soup
print(soup_mars_imag.prettify())

<html class="">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
  <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <title>
   Space Image
  </title>
  <style type="text/css">
   .fancybox-margin{margin-right:17px;}
  </style>
 </head>
 <body>
  <div class="header">
   <nav class="navbar navbar-expand-lg">
    <a class="navbar-brand" href="#">
     <img id="logo" src="image/nasa.png"/>
     <span class="logo">
      Jet Propulsion Laboratory
     </span>
     <span class="logo1">
      California Institute of Technology
     </span>
    </a>
    <button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle

In [9]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url 
    # string to a variable called `featured_image_url`.  Make sure to find the image url to the full size `.jpg` image.
    # reference C:\Users\jjel0\OneDrive\class_repo\gt-virt-atl-data-pt-06-2021-u-c-2\12-Web-Scraping-and-Document-Databases\2\Activities\07-Ins_Splinter
    # Example:
        # featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

        # Collect the latest News Title and Paragraph Text.
featured_image = soup_mars_imag.find('a', class_='showimg fancybox-thumbs')
href = featured_image['href']
featured_image_url = 'https://spaceimages-mars.com/' + href
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


### Mars Facts

In [10]:
# Visit the Mars Facts webpage [here](https://galaxyfacts-mars.com) and use Pandas to scrape the table containing
    # facts about the planet including Diameter, Mass, etc.
url_mars_facts = 'https://galaxyfacts-mars.com'
browser.visit(url_mars_facts)
html_mars_facts = browser.html

In [11]:
# Create a Beautiful Soup object
soup_mars_facts = bs(html_mars_facts, 'lxml')
print(soup_mars_facts.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <title>
   Mars Facts - Interesting facts about Planet Mars
  </title>
 </head>
 <body>
  <nav class="navbar navbar-expand-lg navbar-light fixed-top">
   <div class="container">
    <a class="navbar-brand" href="#">
     <img src="image/logo.png"/>
    </a>
    <button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-bs-target="#navbarNav" data-bs-toggle="collapse" type="button">
     <span class="navbar-toggler-icon">
     </span>
    </button>
    <div class="col

In [17]:
# Use Pandas to convert the data to a HTML table string.
html_mars_facts_df = pd.read_html(url_mars_facts)
html_mars_facts_df

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

### Mars Hemispheres

In [13]:
# Visit the astrogeology site [here](https://marshemispheres.com/) to obtain high resolution images for each of 
    # Mars hemispheres.
url_mars_hemis = 'https://marshemispheres.com/'
browser.visit(url_mars_hemis)
html_mars_hemis = browser.html

In [14]:
# Use BeautifulSoup to help find and parse out the necessary data.  Use either html.parser or lxml parser.
# Create a Beautiful Soup object
soup_mars_hemis = bs(html_mars_hemis, 'html.parser')

# Print formatted version of the soup
print(soup_mars_hemis.prettify())

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | GUSS Astrogeology Science Center
  </title>
  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="css/main.css" media="screen" rel="stylesheet"/>
  <link href="css/print.css" media="print" rel="stylesheet"/>
  <link href="#" rel="icon" type="image/x-ico"/>
 </head>
 <body id="results">
  <header>
   <a href="#" style="float:right;margin-top:10px;" target="_blank">
    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_

In [15]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full
    # resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing
    # the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.


# Append the dictionary with the image url string and the hemisphere title to a list. This list will contain
    # one dictionary for each hemisphere.


# # Example:
    # hemisphere_image_urls = [
    #     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
    #     {"title": "Cerberus Hemisphere", "img_url": "..."},
    #     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    #     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
    # ]
